In [31]:
import json
import pandas as pd
from utils import prep_ans
import numpy as np
from vqa_question_type_processing import question_type_processing as vqa_question_type_processing
from simpsons_question_type_processing import question_type_processing as simpsons_question_type_processing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from io import BytesIO
from sklearn.metrics import accuracy_score
from PIL import Image
import numpy as np
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import wandb

In [32]:
simpsons_anno_paths = ["/home/ngoc/data/simpsons/v1_Annotation_Test_Private_110496_simpsons_vqa.json", "/home/ngoc/data/simpsons/v1_Annotation_Val_simpsons_vqa.json"]

In [33]:
vqa_ans_to_idx, idx_to_vqa_ans, question_type_map = json.load(open("/home/ngoc/githubs/chatgpt-hie-vqa/datasets/answer_dicts_simpsons.json", 'r'))

In [34]:
simpsons_question_type_dict = {}
with open("/home/ngoc/githubs/chatgpt-hie-vqa/result_analysis/train_question_type_gpt_v6_simpsons_1104.json", 'r') as file:
    for line in file:
        question_object = json.loads(line)
        question_str = question_object["question"]
        question_type = question_object["question_type"]
        question_type = vqa_question_type_processing(question_type)
        simpsons_question_type_dict[question_str] = question_type
file.close()

In [35]:

vqa_question_paths = ["/home/ngoc/data/simpsons/v1_Question_Val_simpsons_vqa.json", "/home/ngoc/data/simpsons/v1_Question_Test_Private_110496_simpsons_vqa.json"]
vqa_questions = []
for vqa_question_path in vqa_question_paths:
    with open(vqa_question_path, 'r') as file:
        vqa_questions += json.load(file)["questions"]
        
        
vqa_processed_questions = {}
for question in vqa_questions:
    vqa_processed_questions[question["id"]] = simpsons_question_type_dict[question["question"]]


In [36]:
simpsons_answers = []
for simpsons_anno_path in simpsons_anno_paths:
    with open(simpsons_anno_path, 'r') as file:
        simpsons_answers += json.load(file)["annotations"]

In [37]:
simpsons_processed_answers = {}
for simpsons_answer in  simpsons_answers:
    question_id = simpsons_answer["id"]
    
    answer =  prep_ans(simpsons_answer['answer'])
    
    if answer in list(vqa_ans_to_idx.keys()):
        ans_count = 0
        for judge in simpsons_answer["judgements"].values():
            if judge["answer"] == 1:
                ans_count += 1
        if ans_count >= 2:
            simpsons_processed_answers[question_id] = {}
            simpsons_processed_answers[question_id]["answer"] = answer
            simpsons_processed_answers[question_id]["question_type"] = vqa_processed_questions[question_id]


In [38]:
len(simpsons_processed_answers)

20257

In [39]:
val_result = pd.read_csv("val_predictions.csv")
test_result = pd.read_csv("test_predictions.csv")


In [40]:
test_result

,question_id,vqa_prediction,question_prediction
0,117817,dog,object
1,153452,round,object
2,153763,dress,object
3,130601,2,number
4,10620,3,number
...,...,...,...
28916,163459,sign,object
28917,97819,livingroom,location
28918,146857,right,location
28919,27221,yes,yes/no


In [41]:
def map_answer_scores(question_id):
    if question_id in simpsons_processed_answers.keys():
        answer_dict = simpsons_processed_answers[question_id]
        return answer_dict["answer"]
    else:
        return -1
def map_question_scores(question_id):
    if question_id in simpsons_processed_answers.keys():
        answer_dict = simpsons_processed_answers[question_id]
        return answer_dict["question_type"]
    else:
        return -1

In [42]:
test_result["vqa_label"] = test_result["question_id"].apply(lambda x: map_answer_scores(x))
test_result

,question_id,vqa_prediction,question_prediction,vqa_label
0,117817,dog,object,-1
1,153452,round,object,-1
2,153763,dress,object,-1
3,130601,2,number,-1
4,10620,3,number,3
...,...,...,...,...
28916,163459,sign,object,-1
28917,97819,livingroom,location,livingroom
28918,146857,right,location,-1
28919,27221,yes,yes/no,yes


In [43]:
test_result = test_result[test_result["vqa_label"] != -1]

In [44]:
test_result["question_label"] = test_result["question_id"].apply(lambda x: map_question_scores(x))

/tmp/ipykernel_521724/299494027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_result["question_label"] = test_result["question_id"].apply(lambda x: map_question_scores(x))


In [45]:
# test_result["prediction"] = test_result["prediction"].astype(str)
# test_result["label"] = test_result["label"].astype(str)

In [46]:
test_overall_accuracy  = accuracy_score(test_result["vqa_prediction"], test_result["vqa_label"])
test_overall_accuracy

0.6060230629943772

In [47]:
sorted_unique_labels = ["yes/no", "object", "color", "number", "location","human", "action", "sport", "other"]

In [48]:
sub_accurancy = {}
for question_type_value in sorted_unique_labels:
    sub_test_result = test_result[test_result["question_label"] == question_type_value]
    sub_accuracy_i = accuracy_score(sub_test_result["vqa_prediction"], sub_test_result["vqa_label"])
    sub_accurancy["test_vqa_accuracy_" + question_type_value] = sub_accuracy_i

In [49]:
sub_accurancy

{'test_vqa_accuracy_yes/no': 0.8603542234332425,
 'test_vqa_accuracy_object': 0.3763440860215054,
 'test_vqa_accuracy_color': 0.4199395770392749,
 'test_vqa_accuracy_number': 0.6505555555555556,
 'test_vqa_accuracy_location': 0.42065009560229444,
 'test_vqa_accuracy_human': 0.4342105263157895,
 'test_vqa_accuracy_action': 0.7569593147751607,
 'test_vqa_accuracy_sport': 1.0,
 'test_vqa_accuracy_other': 0.6}

In [50]:
run_id = "kspte150"

In [51]:

wandb.init(project="VQA new", id=run_id, resume="must")

In [52]:
wandb.log({"test_vqa_accuracy": test_overall_accuracy})

In [53]:
wandb.log(sub_accurancy)

In [54]:
test_question_type_accuracy = classification_report(test_result["question_prediction"], test_result["question_label"], output_dict=True)

In [55]:
new_test_question_type_accuracy = {}
for key in test_question_type_accuracy:
    new_test_question_type_accuracy["question_type_" + key] = test_question_type_accuracy[key]
del(test_question_type_accuracy)

In [56]:
wandb.log(new_test_question_type_accuracy)

In [57]:
y_true = test_result['question_label']
y_pred = test_result['question_prediction']
sorted_unique_labels = ["yes/no", "object", "color", "number", "location","human", "action", "sport", "other"]

conf_matrix = confusion_matrix(y_true, y_pred, labels=sorted_unique_labels)
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(20, 9))
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix, annot=True, fmt='d', 
            xticklabels=sorted_unique_labels, yticklabels=sorted_unique_labels, 
            cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Type')
plt.ylabel('True Type')
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', 
            xticklabels=sorted_unique_labels, yticklabels=sorted_unique_labels, 
            cmap='Blues', cbar=False)
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Type')
plt.ylabel('True Type')
plt.suptitle(f'Test confusion matrix')
buffer = BytesIO()
plt.savefig(buffer, format='png')
buffer.seek(0)
image = Image.open(buffer)
image_array = np.array(image)
wandb.log({"Test Confusion Matrix": wandb.Image(image_array)})
plt.close()
plt.show()

In [58]:
wandb.finish()

wandb: WARNING Ensure read and write access to run files dir: /home/ngoc/githubs/chatgpt-hie-vqa/result_analysis/wandb/run-20240324_235451-kspte150/files, control this via the WANDB_DIR env var. See https://docs.wandb.ai/guides/track/environment-variables


question_type_accuracy,▁
test_vqa_accuracy,▁
test_vqa_accuracy_action,▁
test_vqa_accuracy_color,▁
test_vqa_accuracy_human,▁
test_vqa_accuracy_location,▁
test_vqa_accuracy_number,▁
test_vqa_accuracy_object,▁
test_vqa_accuracy_other,▁
test_vqa_accuracy_sport,▁
test_vqa_accuracy_yes/no,▁
